## Retrieve futures market data from Coinglass.com
Currently getting the following:
 - Average Funding rate
 - Total Open Interest

In [1]:
from functions import *

import plotly.graph_objects as go
import datetime
from plotly.subplots import make_subplots
from numerize import numerize
from IPython.display import Image
from IPython.core.display import HTML 

In [2]:
df_usdt = get_df_from_url('fr_usdt')
df_token = get_df_from_url('fr_token')
df_oi = get_df_from_url('oi')

df_usdt = get_weighted_mean_funding_rate(df_usdt, df_oi)
df_token = get_weighted_mean_funding_rate(df_token, df_oi)


In [3]:
# FUNDING RATE
mean_funding_rate_usdt = round(df_usdt['mean_funding_rate'].tail(1)[0], 4)
mean_funding_rate_token = round(df_token['mean_funding_rate'].tail(1)[0], 4)

mean_funding_rate = round((df_usdt['mean_funding_rate'].tail(1)[0] + df_token['mean_funding_rate'].tail(1)[0]) / 2, 4)

# PREDICTED FUNDING RATE
mean_predicted_funding_rate_usdt = round(df_usdt['mean_predicted_funding_rate'].tail(1)[0], 4)
mean_predicted_funding_rate_token = round(df_token['mean_predicted_funding_rate'].tail(1)[0], 4)

mean_predicted_funding_rate = round((df_usdt['mean_predicted_funding_rate'].tail(1)[0] + df_token['mean_predicted_funding_rate'].tail(1)[0]) / 2, 4)

# OPEN INTEREST
total_oi = df_oi['total_open_interest'].tail(1)[0]

# PRINT
print('Funding rate mean (USD or USDT): {}'.format(mean_funding_rate_usdt))
print('Funding rate mean (Token): {}'.format(mean_funding_rate_token))
print('Funding rate mean: {} \n'.format(mean_funding_rate))

#print('Predicted funding rate mean (USD or USDT): {}'.format(mean_predicted_funding_rate_usdt))
#print('Predicted funding rate mean (Token): {}'.format(mean_predicted_funding_rate_token))
print('Predicted funding rate mean: {} \n'.format(mean_predicted_funding_rate))

print('Current Total Open Interest: {}\n'.format(numerize.numerize(total_oi)))

Funding rate mean (USD or USDT): 0.0072
Funding rate mean (Token): 0.0072
Funding rate mean: 0.0072 

Predicted funding rate mean: 0.0036 

Current Total Open Interest: 14.03B



# Charts

In [4]:
# Create series with average funding rate: (usdt + token) / 2
se_mean_funding_rate = round((df_token['mean_funding_rate'] + df_usdt['mean_funding_rate']) / 2, 4)
se_weighted_mean_funding_rate = round((df_token['weighted_mean_funding_rate'] + df_usdt['weighted_mean_funding_rate']) / 2, 4)


# date filter, gets date from 20 days ago
days = 30
date_filter = (datetime.datetime.now() - datetime.timedelta(days = days)).strftime('%Y-%m-%d')

# Create fig with 3 subplots, 1 for price, 1 for funding rate, 1 for open interest
fig = make_subplots(
    rows=3, 
    cols=1,
    subplot_titles=('Price', 'Average Funding Rate', 'Open Interest'),
    shared_xaxes=True,
    vertical_spacing=0.01
)

# Add price plot
fig.add_trace(go.Scatter(x=df_token[date_filter:].index, 
                         y=df_token[date_filter:]['price'],
                         name="Price",
                         mode='lines+markers'
                         ),
              row=1,
              col=1
             )

# Add funding rate plot
#fig.add_trace(go.Scatter(x=df_token[date_filter:].index, 
#                         y=se_mean_funding_rate[date_filter:],
#                         name="Average Funding Rate",                         
#                         mode='lines+markers'),
#              row=2,
#              col=1
#             )
fig.add_trace(go.Bar(x=df_token[date_filter:].index, 
                         y=se_weighted_mean_funding_rate[date_filter:],
                         name="Average Funding Rate",                         
                        ),
              row=2,
              col=1
             )

fig.add_hline(y=0, row=2, col=1, line_dash="dot", opacity=0.3)
fig.add_hline(y=0.01, row=2, col=1, line_dash="dot", opacity=0.3, annotation_text='Baseline')

# Add open interest plot
fig.add_trace(go.Scatter(x=df_oi[date_filter:].index, 
                         y=df_oi[date_filter:]['total_open_interest'],
                         name="Total Open Interest",                       
                         mode='lines+markers'),
              row=3,
              col=1
             )

###################
# Add annotations #
###################

# Add annotation to current price
fig.add_annotation(x=df_token[date_filter:].index[-1], 
                   y=df_token[date_filter:]['price'][-1],
                   text='<b>{}</b>'.format(round(df_token[date_filter:]['price'][-1])),
                   showarrow=False,
                   xanchor='left',
                   yanchor='bottom',
                   row=1, col=1)


# Add annotation to current funding rate
#fig.add_annotation(x=df_token[date_filter:].index[-1], 
#                   y=se_mean_funding_rate[date_filter:][-1],
#                   text='<b>{}%</b>'.format(se_mean_funding_rate[-1]),
#                   showarrow=False,
#                   xanchor='left',
#                   yanchor='bottom',
#                   row=2, col=1)

fig.add_annotation(x=df_token[date_filter:].index[-1], 
                   y=se_weighted_mean_funding_rate[date_filter:][-1],
                   text='<b>{}%</b>'.format(se_weighted_mean_funding_rate[-1]),
                   showarrow=False,
                   xanchor='left',
                   yanchor='bottom',
                   row=2, col=1)


# Add annotation to current open interest
fig.add_annotation(x=df_oi[date_filter:].index[-1], 
                   y=df_oi[date_filter:]['total_open_interest'][-1],
                   text='<b>{}</b>'.format(numerize.numerize(df_oi[date_filter:]['total_open_interest'][-1])),
                   showarrow=False,
                   xanchor='left',
                   yanchor='bottom',
                   row=3, col=1)


# Add Title and height
fig.update_layout(legend=dict(y=0.5, font_size=12), 
                  height=1200, 
                  width=1500,
                  title='<B>Futures Market Data (last {} days)</B>'.format(days),
                  title_x=0.5)


fig.show()




# Hourly funding rate 

In [5]:
df = read_csv_sftp('IP_ADDRESS', 'gustavo', '/home/gustavo/btc_futures_data/hour_funding_rate.csv')
#df = pd.read_csv('hour_funding_rate.csv')

df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.tz_localize('UTC').dt.tz_convert('Europe/Budapest')

df.set_index('date', inplace=True)

#df.head()

mask_usdt = df['type'] == 'USDT'
mask_token = df['type'] == 'TOKEN'

se_usdt_weighted_mean_funding_rate = df[mask_usdt]['weighted_mean_funding_rate']
se_token_weighted_mean_funding_rate = df[mask_token]['weighted_mean_funding_rate']

se_weighted_mean_funding_rate = round((se_token_weighted_mean_funding_rate + se_usdt_weighted_mean_funding_rate) / 2, 4)


# Chart config

# date filter, gets date from 20 days ago
days = 20
date_filter = (datetime.datetime.now() - datetime.timedelta(days = days)).strftime('%Y-%m-%d')

# Instantiate fig with 2 y axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add price plot
fig.add_trace(go.Scatter(x=df[mask_usdt][date_filter:].index, 
                         y=df[mask_usdt][date_filter:]['price'],
                         name="Price",
                         mode='lines+markers',
                         ),
                         secondary_y=True,
                         
             )

# Add funding rate plot
fig.add_trace(go.Bar(x=df[mask_usdt][date_filter:].index, 
                         y=se_weighted_mean_funding_rate[date_filter:],
                         name="Weighted Average Funding Rate"            
                        ),
                        secondary_y=False
                        
             )

# Change axis ranges
fig.update_yaxes(range=[df['price'].min() - 500 , df['price'].max() + 500], secondary_y=True)
fig.update_yaxes(range=[-0.02, 0.02], secondary_y=False)

# Add horizontal line
fig.add_hline(y=0.01, line_dash="dot", opacity=0.3, annotation_text='Baseline')

# Change color of primary y axis
fig.update_traces(marker_color='yellowgreen', secondary_y=False)
fig.update_traces(marker_color='black', secondary_y=True)

# Add annotation to current funding rate
fig.add_annotation(x=df[mask_usdt][date_filter:].index[-1], 
                   y=se_weighted_mean_funding_rate[date_filter:][-1],
                   text='<b>{}%</b>'.format(se_weighted_mean_funding_rate[-1]),
                   showarrow=False,
                   xanchor='left',
                   yanchor='bottom',
                   )


# Add Title and height
fig.update_layout(legend=dict(y=0.5, font_size=12), 
                  height=1200, 
                  width=1800,
                  title='<B>Hourly Funding Rate Data (last {} days)</B>'.format(days),
                  title_x=0.5,
                  )



fig.show()

In [6]:
df = read_csv_sftp('IP_ADDRESS', 'gustavo', '/home/gustavo/btc_futures_data/minute_funding_rate.csv')
#df = pd.read_csv('minute_funding_rate.csv')

df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.tz_localize('UTC').dt.tz_convert('Europe/Budapest')

df.set_index('date', inplace=True)

#df.head()

mask_usdt = df['type'] == 'USDT'
mask_token = df['type'] == 'TOKEN'

se_usdt_weighted_mean_funding_rate = df[mask_usdt]['weighted_mean_funding_rate']
se_token_weighted_mean_funding_rate = df[mask_token]['weighted_mean_funding_rate']

se_weighted_mean_funding_rate = round((se_token_weighted_mean_funding_rate + se_usdt_weighted_mean_funding_rate) / 2, 4)


# Chart config

# date filter
h = 24
date_filter = (datetime.datetime.now() - datetime.timedelta(hours = h)).strftime('%Y-%m-%d-%H')

# Instantiate fig with 2 y axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add price plot
fig.add_trace(go.Scatter(x=df[mask_usdt][date_filter:].index, 
                         y=df[mask_usdt][date_filter:]['price'],
                         name="Price",
                         mode='lines+markers',
                         ),
                         secondary_y=True,
                         
             )

# Add funding rate plot
fig.add_trace(go.Bar(x=df[mask_usdt][date_filter:].index, 
                         y=se_weighted_mean_funding_rate[date_filter:],
                         name="Weighted Average Funding Rate"            
                        ),
                        secondary_y=False
                        
             )

# Change axis ranges
#fig.update_yaxes(range=[df[date_filter]['price'].min() - 500 , df['price'][date_filter].max() + 500], secondary_y=True)
fig.update_yaxes(range=[-0.015, 0.015], secondary_y=False)
#fig.update_yaxes(range=[df[date_filter]['weighted_mean_funding_rate'].min() - 0.01 , df[date_filter]['weighted_mean_funding_rate'].max() + 0.01], secondary_y=False)

# Add horizontal line
fig.add_hline(y=0.01, line_dash="dot", opacity=0.3, annotation_text='Baseline')

# Change color of primary y axis
fig.update_traces(marker_color='yellowgreen', secondary_y=False)
fig.update_traces(marker_color='black', secondary_y=True)

# Add annotation to current funding rate
fig.add_annotation(x=df[mask_usdt][date_filter:].index[-1], 
                   y=se_weighted_mean_funding_rate[date_filter:][-1],
                   text='<b>{}%</b>'.format(se_weighted_mean_funding_rate[-1]),
                   showarrow=False,
                   xanchor='left',
                   yanchor='bottom',
                   )


# Add Title and height
fig.update_layout(legend=dict(y=0.5, font_size=12), 
                  height=1200, 
                  width=1800,
                  title='<B>5 Minute Funding Rate Data (last {} hours)</B>'.format(h),
                  title_x=0.5,
                  )



fig.show()

# Get current fear and greed
Image(url= "https://alternative.me/crypto/fear-and-greed-index.png", width=400, height=400)